# Data Pre-processing

## A. Data Entry

In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import utils
from scipy.stats import skew
import math
import re
from collections import deque

In [8]:
# load the data set
df = pd.read_csv("/Users/ibulmnie/Documents/GitHub/ML20222.PredictionBitcoin/ML20222.PredictionBitcoin/data/raw_data.csv")

ParserError: Error tokenizing data. C error: Expected 17 fields in line 14, saw 18


In [ ]:
df.head()

In [ ]:
rows, cols = df.shape
print("There are {} rows.".format(rows))
print("There are {} columns.".format(cols))

In [ ]:
# check data type
df.info(verbose = True)

## B. Data Preprocessing

This section focuses on enhancing the data quality by finding the caveats of the data set and manipulating and dropping data before feeding them into the model's input pipeline. 
It also involves feature selection methods aimed at eliminating features that have the least impact on the target variable.

## 1. Evaluating Target Variables

Since the objective is to predict the price of Bitcoin, which is represented by the "BTC_close" variable, it is essential to ensure that "BTC_close" is valid for all records. 
Otherwise, if there are missing or invalid values in the "BTC_close" variable, it would result in the absence of a target variable for the predictive model.

In [ ]:
df['BTC_close'].describe()

In [ ]:
print("Number of records with NULL price: ", df['BTC_close'].isna().sum())

In [ ]:
# collect the row indices where pice is NULL
price_isna_idx = np.where(df['BTC_close'].isna()==True)[0]
# remove the rows where target price is not available 
df.drop(price_isna_idx, axis=0, inplace=True)
print("Number of records with NULL price: ", df['BTC_close'].isna().sum())
print("-"*50)
rows, cols = df.shape
print("There are {} rows.".format(rows))
print("There are {} columns.".format(cols))

Before clipping the values of the target variables

In [ ]:
sns.set_theme()
price_dist = sns.displot(df['BTC_close'], kde=True)
price_dist.fig.set_size_inches(8,5)
print("Skewness: ", df['BTC_close'].skew())

?????? Các giá trị ngoại lai có thể mang thông tin quan trọng hoặc đại diện cho các biến động đặc biệt trong thị trường Bitcoin. Nếu xóa các giá trị này, có thể mất đi một phần quan trọng của thông tin và dẫn đến kết quả dự đoán không chính xác. => không xoá nhỉ
Lý do: 
1. Quan sát biến động cao: Giá Bitcoin có thể biến động mạnh và có những giai đoạn tăng giá đáng kể. Các giá trị ngoại lai có thể đại diện cho những biến động này và mang thông tin quan trọng về sự biến động của thị trường Bitcoin. Bằng cách giữ lại các giá trị ngoại lai, bạn có thể cho phép mô hình học hỏi và ứng phó với những biến động tương tự trong tương lai.

2. Quan tâm đến sự biến đổi đặc biệt: Các giá trị ngoại lai có thể đại diện cho các sự kiện đặc biệt hoặc các yếu tố độc đáo trong thị trường Bitcoin. Bằng cách giữ lại các giá trị này, bạn có thể cho phép mô hình nhận biết và tìm hiểu về các yếu tố đặc biệt này, giúp cải thiện khả năng dự đoán trong tương lai.

3. Tiềm năng lợi nhuận cao: Vì giá Bitcoin có thể tăng lên rất cao, việc giữ lại các giá trị ngoại lai có thể mang lại tiềm năng lợi nhuận cao trong trường hợp mô hình dự đoán chính xác các biến động này. Nếu bạn xóa các giá trị ngoại lai, bạn có thể mất đi cơ hội tận dụng những biến động giá trị Bitcoin tiềm năng.

4. Quan sát các xu hướng dài hạn: Bằng cách giữ lại các giá trị ngoại lai, bạn có thể giúp mô hình nhận biết và quan sát các xu hướng dài hạn của giá Bitcoin. Điều này có thể cung cấp thông tin giá trị về sự phát triển và tiềm năng tương lai của Bitcoin, giúp cải thiện khả năng dự đoán.

5. Tính chất đặc biệt của Bitcoin: Bitcoin có tính chất độc đáo và có thể trải qua các biến động rất lớn trong giá trị. Điều này có thể dẫn đến sự lệch trong biểu đồ khi các giá trị ngoại lai biểu thị những biến động này. Biểu đồ lệch có thể phản ánh tính chất không thường xuyên và không đồng nhất của dữ liệu Bitcoin.


## 2. Handling NULL Values

#### a. Dealing with the datetime feature

In [ ]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

#### b. NULL values

In [ ]:
null_count_arr = df.isna().sum()
print(null_count_arr.sort_values(ascending=False))

In [ ]:
# plot the distribution of missing values by 24 features 
null_ratio = (null_count_arr.sort_values(ascending=False)[:24]/df.shape[0])*100
plt.figure(figsize=(15,8))
plt.xticks(rotation='vertical')
sns.barplot(x = null_ratio.index, y = null_ratio)
plt.xlabel('Features', fontsize = 15)
plt.ylabel('Missing Values (%)', fontsize = 5)
plt.title('Percentage of Missing Values by Features', fontsize = 15)

By observing the above information, we realize there are two problems with NULL data.

Firstly: Market Index including (S&P500, DJI, JP225) and Commodities (Gold, Silver, Copper) have no transactions on Saturday, and Sunday. If you look at our data set, they are all NULL on Saturdays and Sundays.

----> Solution: fill in the null value of Saturday, Sunday using the Friday of the last trading session of that week

Second: Other Cryptocurrencies (DOGE, LTC, ETH, XRP) have a NULL price because it is newly released later, specifically as follows:
XRP : 22/01/2015
ETH : 10/03/2016
LTC : April 28, 2016
DOGE : 03/06/2017
----> Solution: fill null value with 0 representing the non-occurrence

#### The first problem

In [ ]:
# Get the earliest date by excluding missing values
earliest_date = df['Date'].dropna().min()

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    date = row['Date']
    # Check if the date value is missing (NaN)
    if pd.isnull(date):
        # Assign the earliest date to the missing value
        df.at[index, 'Date'] = earliest_date

In [ ]:
df.head()

#### The second problem

In [ ]:
# # Fill in the missing data with 0 value
df_filled = df.fillna(0)


In [ ]:
# Check existence of NaN value in DataFrame
print("Is There any 'NaN' value: ", df_filled.isnull().values.any())

# Check existence of duplicate values in DataFrame
print("Is there any duplicate value: ", df_filled.index.duplicated().any())


## 3. Correlation Analysis

In [ ]:
ds = df.set_index('Date') 
sns.heatmap(ds.corr(), cmap="RdBu")

In [ ]:
# Retrieve the number of categorical fields and numerical fields
# field indices where column is categorical
categorical_col_idx = np.where(ds.dtypes == object)[0]
# field indices where column is numerical
numerical_col_idx = np.where(ds.dtypes != object)[0]
print("Number of categorical fields: ",categorical_col_idx.shape[0])
print("Number of numerical fields: ",numerical_col_idx.shape[0])

## 4. Finding outliers

In [ ]:
continuous_features = df.drop("Date", axis=1).columns.tolist()

In [ ]:
for feature in continuous_features:
    df_copy = df.copy()
    df_copy[feature] = np.log1p(df_copy[feature])
    df_copy.boxplot(column=feature)
    plt.ylabel(feature)
    plt.show()

## 5. Feature Engineering

#### a. Preparing X and y to do train test split

In [ ]:
X = df_filled.iloc[:, 1:]
y = df_filled.iloc[:, 0]
print(X.shape, y.shape)

#### b. The sample of the training and test files for the models

###### IF the way you divide is to use the date call to meet the set ratio

Train, Test = 85, 15

In [ ]:
"""from datetime import datetime

for index in total.index:
    total.loc[index, "Date"] = datetime.strptime(str(total.loc[index, "Date"])[:10], '%Y-%m-%d').date()

Start_day = date(2016, 3, 10)
Test_day = date(2022,3,1)
End_day = date(2023,4,16)


# train, test
total = total[(total["Date"] >= Start_day) & (total["Date"] <= End_day)].reset_index(drop = True)
train_dataset = total[total["Date"] < Test_day].reset_index(drop = True)
test_dataset = total[total["Date"] >= Test_day].reset_index(drop = True)"""

In [ ]:
"""X_train = train_dataset.drop(["Date"], axis=1)[:-1]
y_train = train_dataset["BTC_close"][1:].reset_index(drop=True)

X_test = test_dataset.drop(["Date"], axis=1)[:-1]
y_test = test_dataset["BTC_close"][1:].reset_index(drop=True)"""

In [ ]:
"""test_ratio = len(test_dataset) / len(total)

print("Test_data/total:", test_ratio)"""

In [ ]:
"""print("X_train size:", X_train.shape)
print("Y_train size:", y_train.shape)
print("X_test size:", X_test.shape)
print("Size y_test:", y_test.shape)"""

###### IF the way you split is to use train_test_split from Sklearn library



Train, Valid, Test = 70,15,15

In [ ]:
"""from sklearn.model_selection import train_test_split"""

In [ ]:
"""X_train, X_test, y_train, y_testn = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=False)"""

## 6. Feature Selection

Using the "Extra Trees Regressor" tool of the Scikitlearn library to create a graph that visually shows the importance of the independent variables X to the dependent variable Y in the forecasting model

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

feat_selection = ExtraTreesRegressor()
feat_selection.fit(X,y)

In [ ]:
print(feat_selection.feature_importances_)

In [ ]:
#plot a graph of feature importances
feat_importances = pd.Series(feat_selection.feature_importances_, index=X.columns)
feat_importances.nlargest(23).plot(kind='barh')
plt.show()

But for our case the number of features are less and our purpose so we will use all of them.

## 7. Feature Scaling

## C. Finishing Touch

In [ ]:
# save preprocessed data in .csv format for model building
#df_filled.to_csv('data.csv', index=False)